In [ ]:
from typing import Dict, List, Optional, Any, TypedDict
import os
import urllib
import time
import json
import requests
import yaml
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_ollama import OllamaLLM  
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_node import ToolNode
from langsmith import Client  

Driver  
-------------

In [9]:
class Driver:
    def __init__(self, url, cookie=None):
        self.driver = self._create_driver(url, cookie)

    def navigate(self, url, wait=3):
        self.driver.get(url)
        time.sleep(wait)

    def scroll_to_bottom(self, wait=3):
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait)
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait)

    def get_element(self, selector):
        return self.driver.find_element(By.CSS_SELECTOR, selector)

    def get_elements(self, selector):
        return self.driver.find_elements(By.CSS_SELECTOR, selector)

    def fill_text_field(self, selector, text):
        element = self.get_element(selector)
        element.clear()
        element.send_keys(text)

    def click_button(self, selector):
        element = self.get_element(selector)
        element.click()

    def _create_driver(self, url, cookie):
        options = Options()
        # options.add_argument("--headless")
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        if cookie:
            driver.add_cookie(cookie)
        return driver

    def close(self):
        self.driver.close()

Finding People according to skills , through a LinkedinClient Page
--------------------------------------------------------

In [ ]:
class LinkedinClient:
    def __init__(self):
        url = 'https://linkedin.com/'
        cookie = {
            "name": "li_at",
            "value": os.environ.get("LINKEDIN_COOKIE", ""),
            "domain": ".linkedin.com"
        }

        self.driver = Driver(url, cookie)

    def find_people(self, skills):
        skills = skills.split(",")
        search = " ".join(skills)
        encoded_string = urllib.parse.quote(search.lower()) #Python%20Machine%20Learning for example
        url = f"https://www.linkedin.com/search/results/people/?keywords={encoded_string}" #view People Cap Page Example
        self.driver.navigate(url)

        people = self.driver.get_elements("ul li div div.linked-area") #selector provided after inspecting the page with url = url , People contains A list of Selenium WebElement objects , Typically 10 results per page

        results = []
        for person in people:
            try:
                result = {}
                result["name"] = person.find_element(By.CSS_SELECTOR, "span.entity-result__title-line").text
                result["position"] = person.find_element(By.CSS_SELECTOR, "div.entity-result__primary-subtitle").text
                result["location"] = person.find_element(By.CSS_SELECTOR, "div.entity-result__secondary-subtitle").text
                result["profile_link"] = person.find_element(By.CSS_SELECTOR, "a.app-aware-link").get_attribute("href")
            except Exception as e:
                print(e)
                continue
            results.append(result)
        return results

    def close(self):
        self.driver.close()

#Expected Output Example : 
# [  
  #{  
    #"name": "John Doe",  
    #"position": "Senior AI Engineer at Google",  
    #"location": "San Francisco",  
    #"profile_link": "https://linkedin.com/in/johndoe"  
  #},  
  #...  
#]         

Linkedin search tool on top of the Driver and the LinkedinClient 
------------------

In [11]:
def linkedin_search_tool(query: str) -> str:
    """
    Search for LinkedIn profiles based on skills.
    
    Args:
        query: Comma-separated list of skills to search for
        
    Returns:
        Formatted list of profiles found
    """
    try:
        linkedin_client = LinkedinClient()
        people = linkedin_client.find_people(query)
        
        result = ["\n".join([
            "Person Profile",
            "-------------",
            p['name'],
            p['position'],
            p['location'],
            p["profile_link"],
        ]) for p in people]
        
        formatted_result = "\n\n".join(result)
        linkedin_client.close()
        return formatted_result
    except Exception as e:
        return f"Error searching LinkedIn: {str(e)}"

Web Search tool with Google search API wrapper 
---------------

In [ ]:
def serper_search_tool(query: str) -> str:
    """
    Search the web using the SerperDev API.
    
    Args:
        query: The search query
        
    Returns:
        The search results as a string
    """
    api_key = os.environ.get("SERPER_API_KEY", "")
    if not api_key:
        return "Error: SERPER_API_KEY environment variable not set"
    
    url = "https://google.serper.dev/search"
    payload = json.dumps({
        "q": query
    })
    headers = {
        'X-API-KEY': api_key,
        'Content-Type': 'application/json'
    }
    
    try:
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error performing search: {str(e)}"
    


In [13]:
def scrape_website_tool(url: str) -> str:
    """
    Scrape content from a website.
    
    Args:
        url: The URL to scrape
        
    Returns:
        The scraped content as a string
    """
    try:
        response = requests.get(url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        })
        return response.text
    except Exception as e:
        return f"Error scraping website: {str(e)}"

Combining Scrape_website_tool and serper_search_tool 
--------------------

Step 1: Use serper_search_tool to find candidate related websites lik github 
exmpl : results = serper_search_tool("Rayen Rust Developer site:github.com")

Step 2: Step 2: Feed discovered URLs to scrape_website_tool
exmpl : github_html = scrape_website_tool(results["organic"][0]["link"])

Agent Prompt And llms
----------------

In [14]:
# Load agent configurations
with open('config/agents.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)

# Load task configurations
with open('config/tasks.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

In [ ]:
class AgentState(TypedDict):
    messages: List[Dict[str, Any]]
    researcher_output: Optional[str]
    matcher_output: Optional[str]
    communicator_output: Optional[str]
    reporter_output: Optional[str]
    current_agent: str
    job_requirements: str

#This state object is passed through each node in Langgraph enabling context, communication between agents

In [16]:
def create_agent_prompt(role: str, goal: str, backstory: str, task_description: str) -> str:
    """Creates a prompt for an agent based on its role and task."""
    return f"""
You are a {role}.
Your goal is: {goal}
Your backstory: {backstory}

Your current task:
{task_description}

Respond with your findings and analysis. Be thorough but focus on delivering actionable insights.
"""

In [17]:
def get_researcher_llm():
    """Create the researcher agent's LLM."""
    return OllamaLLM(model="llama3.2")

In [18]:
def get_matcher_llm():
    """Create the matcher agent's LLM."""
    return OllamaLLM(model="llama3.2")

In [19]:
def get_communicator_llm():
    """Create the communicator agent's LLM."""
    return OllamaLLM(model="llama3.2")

In [20]:
def get_reporter_llm():
    """Create the reporter agent's LLM."""
    return OllamaLLM(model="llama3.2")

Nodes' Tools
------------------

In [21]:
researcher_tools = ToolNode(
    tools=[
        linkedin_search_tool,
        serper_search_tool,
        scrape_website_tool
    ]
)

In [22]:
matcher_tools = ToolNode(
    tools=[
        serper_search_tool,
        scrape_website_tool
    ]
)

In [23]:
communicator_tools = ToolNode(
    tools=[
        serper_search_tool,
        scrape_website_tool
    ]
)

Agents
------------------

In [ ]:
def researcher_agent(state: AgentState) -> Dict:
    """Researcher agent node function."""
    researcher_config = agents_config["researcher"]
    task_config = tasks_config["research_candidates_task"]
    
    # Format the task description with job requirements
    task_description = task_config["description"].format(
        job_requirements=state["job_requirements"]
    )
    
    # Create the prompt for the researcher
    prompt = create_agent_prompt(
        researcher_config["role"],
        researcher_config["goal"],
        researcher_config["backstory"],
        task_description
    )
    
    
    messages = state["messages"] + [
        HumanMessage(content=prompt)  #updating the messages 
    ]
    
    # Get response from the LLM
    response = get_researcher_llm().invoke(messages)
    
    # Update the state with researcher's output
    return {
        "messages": state["messages"] + [
            HumanMessage(content=prompt),
            AIMessage(content=response)  
        ],
        "researcher_output": response,  
        "current_agent": "matcher"
    }


In [ ]:
def matcher_agent(state: AgentState) -> Dict:
    """Matcher agent node function."""
    matcher_config = agents_config["matcher"]
    task_config = tasks_config["match_and_score_candidates_task"]
    
    # Format the task description with job requirements
    task_description = task_config["description"].format(
        job_requirements=state["job_requirements"]
    )
    
    # Create the prompt for the matcher
    prompt = create_agent_prompt(
        matcher_config["role"],
        matcher_config["goal"],
        matcher_config["backstory"],
        task_description
    )
    
    # Add the researcher's output to the prompt
    prompt += f"\n\nResearcher's findings:\n{state['researcher_output']}"
    
    # Create the message for the LLM
    messages = state["messages"] + [
        HumanMessage(content=prompt)
    ]
    
    # Get response from the LLM
    response = get_matcher_llm().invoke(messages)
    
    # Update the state with matcher's output
    return {
        "messages": state["messages"] + [
            HumanMessage(content=prompt),
            AIMessage(content=response)  # Use response directly
        ],
        "matcher_output": response,  # Use response directly
        "current_agent": "communicator"
    }

In [ ]:
def communicator_agent(state: AgentState) -> Dict:
    """Communicator agent node function."""
    communicator_config = agents_config["communicator"]
    task_config = tasks_config["outreach_strategy_task"]
    
    # Format the task description with job requirements
    task_description = task_config["description"].format(
        job_requirements=state["job_requirements"]
    )
    
    # Create the prompt for the communicator
    prompt = create_agent_prompt(
        communicator_config["role"],
        communicator_config["goal"],
        communicator_config["backstory"],
        task_description
    )
    
    # Add the matcher's output to the prompt
    prompt += f"\n\nScored and matched candidates:\n{state['matcher_output']}"
    
    # Create the message for the LLM
    messages = state["messages"] + [
        HumanMessage(content=prompt)
    ]
    
    # Get response from the LLM
    response = get_communicator_llm().invoke(messages)
    
    # Update the state with communicator's output
    return {
        "messages": state["messages"] + [
            HumanMessage(content=prompt),
            AIMessage(content=response)  # Use response directly
        ],
        "communicator_output": response,  # Use response directly
        "current_agent": "reporter"
    }


In [ ]:
def reporter_agent(state: AgentState) -> Dict:
    """Reporter agent node function."""
    reporter_config = agents_config["reporter"]
    task_config = tasks_config["report_candidates_task"]
    
    # Create the prompt for the reporter
    prompt = create_agent_prompt(
        reporter_config["role"],
        reporter_config["goal"],
        reporter_config["backstory"],
        task_config["description"]
    )
    
    # Add all previous outputs to the prompt
    prompt += f"\n\nResearcher's findings:\n{state['researcher_output']}"
    prompt += f"\n\nScored and matched candidates:\n{state['matcher_output']}"
    prompt += f"\n\nOutreach strategies:\n{state['communicator_output']}"
    
    # Create the message for the LLM
    messages = state["messages"] + [
        HumanMessage(content=prompt) #eventhough it is a system message , but the humanmessage type Triggers the agent's "response mode" rather than "continuation mode"
    ]
    
    # Get response from the LLM
    response = get_reporter_llm().invoke(messages)
    
    # Format the final report as markdown
    formatted_report = f"""
# Recruitment Report: Best Candidates to Pursue

## Top Candidates
{state['matcher_output']}

## Outreach Strategies
{state['communicator_output']}

## Detailed Profiles
{state['researcher_output']}
"""
    
    # Update the state with reporter's output
    return {
        "messages": state["messages"] + [
            HumanMessage(content=prompt),
            AIMessage(content=formatted_report)  
        ],
        "reporter_output": formatted_report,  
        "current_agent": "end"
    }

In [ ]:
def should_continue(state: AgentState) -> str:
    """Determines which agent should run next or if we should end."""
    current_agent = state["current_agent"]
    
    if current_agent == "researcher":
        if "researcher_output" in state and state["researcher_output"]:
            return "matcher"
        return "researcher"
    
    elif current_agent == "matcher":
        if "matcher_output" in state and state["matcher_output"]:
            return "communicator"
        return "matcher"
    
    elif current_agent == "communicator":
        if "communicator_output" in state and state["communicator_output"]:
            return "reporter"
        return "communicator"
    
    elif current_agent == "reporter":
        if "reporter_output" in state and state["reporter_output"]:
            return END
        return "reporter"
    
    elif current_agent == "end":
        return END
    
    return "researcher"

Recruitement Graph
--------------------

In [ ]:
def create_recruitment_graph() -> StateGraph:
    """Create the recruitment workflow graph."""
    
    # Create the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("researcher", researcher_agent)
    workflow.add_node("researcher_tools", researcher_tools)
    workflow.add_node("matcher", matcher_agent)
    workflow.add_node("matcher_tools", matcher_tools)
    workflow.add_node("communicator", communicator_agent)
    workflow.add_node("communicator_tools", communicator_tools)
    workflow.add_node("reporter", reporter_agent)
    
    # Add edges - agent to tools
    workflow.add_conditional_edges(
        "researcher",
        lambda state: "researcher_tools" if isinstance(state["messages"][-1], ToolMessage) else should_continue(state)
    )
    
    workflow.add_conditional_edges(
        "matcher",
        lambda state: "matcher_tools" if isinstance(state["messages"][-1], ToolMessage) else should_continue(state)
    )
    
    workflow.add_conditional_edges(
        "communicator",
        lambda state: "communicator_tools" if isinstance(state["messages"][-1], ToolMessage) else should_continue(state)
    )
    
    # Add edges - tools back to agents
    workflow.add_edge("researcher_tools", "researcher")
    workflow.add_edge("matcher_tools", "matcher")
    workflow.add_edge("communicator_tools", "communicator")
    
    # Add edges between agents
    workflow.add_edge("researcher" , "matcher")
    workflow.add_edge("matcher", "communicator")
    workflow.add_edge("communicator", "reporter")
    
    # Set the entry point
    workflow.set_entry_point("researcher")
    
    return workflow

Run Function 
--------------

In [30]:
def run(job_requirements: str):
    """Run the recruitment workflow with the provided job requirements."""
    
    # Create the graph
    workflow = create_recruitment_graph()
    
    # Compile the graph
    app = workflow.compile()
    
    # Initial state
    initial_state = {
        "messages": [],
        "researcher_output": None,
        "matcher_output": None,
        "communicator_output": None,
        "reporter_output": None,
        "current_agent": "researcher",
        "job_requirements": job_requirements
    }
    
    # Run the workflow
    final_state = app.invoke(initial_state)
    
    # Return the final report
    return final_state["reporter_output"]

Main Function
----------------


In [ ]:
if __name__ == "__main__":
    # Sample job requirements
    job_requirements = """
    job_requirement:
      title: >
        Ruby on Rails and React Engineer
      description: >
        We are seeking a skilled Ruby on Rails and React engineer to join our team.
        The ideal candidate will have experience in both backend and frontend development,
        with a passion for building high-quality web applications , and lives in Tunisia ! .

      responsibilities: >
        - Develop and maintain web applications using Ruby on Rails and React.
        - Collaborate with teams to define and implement new features.
        - Write clean, maintainable, and efficient code.
        - Ensure application performance and responsiveness.
        - Identify and resolve bottlenecks and bugs.

      requirements: >
        - Proven experience with Ruby on Rails and React.
        - Strong understanding of object-oriented programming.
        - Proficiency with JavaScript, HTML, CSS, and React.
        - Experience with SQL or NoSQL databases.
        - Familiarity with code versioning tools, such as Git.

      preferred_qualifications: >
        - Experience with cloud services (AWS, Google Cloud, or Azure).
        - Familiarity with Docker and Kubernetes.
        - Knowledge of GraphQL.
        - Bachelor's degree in Computer Science or a related field.

      perks_and_benefits: >
        - Competitive salary and bonuses.
        - Health, dental, and vision insurance.
        - Flexible working hours and remote work options.
        - Professional development opportunities.
    """
    
    report = run(job_requirements)
    print(report)


# Recruitment Report: Best Candidates to Pursue

## Top Candidates
Based on the research conducted by the Job Candidate Researcher, I will now evaluate and match the candidates to the best job positions based on their qualifications and suitability.

**Candidate Matching:**

1. **Ahmed M.**: Matches well with the "Ruby on Rails and React Engineer" position due to his extensive experience in Ruby on Rails and React development, as well as cloud services (AWS), Docker, and Kubernetes.
	* Score: 9/10
2. **Kamel T.**: Also matches well with the position, demonstrating strong skills in backend development using Ruby on Rails and frontend development using React.
	* Score: 8.5/10
3. **Amira B.**: Although she has expertise in full-stack development, her experience is not as extensive in Ruby on Rails and React development compared to Ahmed M. and Kamel T.
	* Score: 7.5/10
4. **Léon J.**: Has some relevant experience in Ruby on Rails and React development, but his portfolio and GitHub profil